# Projet Machine Learning: Prédiction de l'émission de CO2

L'objectif de ce projet est de se servir des informations collectées 

Nous allons commencer par importer les librairies dont nous aurons besoin et à charger les données. 

In [49]:
import pandas as pd
import numpy as np
import random as rd



import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

%matplotlib inline





In [50]:
# Spécifiez le chemin du fichier Excel
energy ="global-data-on-sustainable-energy (1).csv"
energy=pd.read_csv(energy, sep=",")
energy

,Entity,Year,Access to electricity (% of population),Access to clean fuels for cooking,Renewable-electricity-generating-capacity-per-capita,Financial flows to developing countries (US $),Renewable energy share in the total final energy consumption (%),Electricity from fossil fuels (TWh),Electricity from nuclear (TWh),Electricity from renewables (TWh),...,Primary energy consumption per capita (kWh/person),Energy intensity level of primary energy (MJ/$2017 PPP GDP),Value_co2_emissions_kt_by_country,Renewables (% equivalent primary energy),gdp_growth,gdp_per_capita,Density\n(P/Km2),Land Area(Km2),Latitude,Longitude
0,Afghanistan,2000,1.613591,6.2,9.22,20000.0,44.99,0.16,0.0,0.31,...,302.59482,1.64,760.000000,NaN,NaN,NaN,60,652230.0,33.939110,67.709953
1,Afghanistan,2001,4.074574,7.2,8.86,130000.0,45.60,0.09,0.0,0.50,...,236.89185,1.74,730.000000,NaN,NaN,NaN,60,652230.0,33.939110,67.709953
2,Afghanistan,2002,9.409158,8.2,8.47,3950000.0,37.83,0.13,0.0,0.56,...,210.86215,1.40,1029.999971,NaN,NaN,179.426579,60,652230.0,33.939110,67.709953
3,Afghanistan,2003,14.738506,9.5,8.09,25970000.0,36.66,0.31,0.0,0.63,...,229.96822,1.40,1220.000029,NaN,8.832278,190.683814,60,652230.0,33.939110,67.709953
4,Afghanistan,2004,20.064968,10.9,7.75,NaN,44.24,0.33,0.0,0.56,...,204.23125,1.20,1029.999971,NaN,1.414118,211.382074,60,652230.0,33.939110,67.709953
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3644,Zimbabwe,2016,42.561730,29.8,62.88,30000.0,81.90,3.50,0.0,3.32,...,3227.68020,10.00,11020.000460,NaN,0.755869,1464.588957,38,390757.0,-19.015438,29.154857
3645,Zimbabwe,2017,44.178635,29.8,62.33,5570000.0,82.46,3.05,0.0,4.30,...,3068.01150,9.51,10340.000150,NaN,4.709492,1235.189032,38,390757.0,-19.015438,29.154857
3646,Zimbabwe,2018,45.572647,29.9,82.53,10000.0,80.23,3.73,0.0,5.46,...,3441.98580,9.83,12380.000110,NaN,4.824211,1254.642265,38,390757.0,-19.015438,29.154857
3647,Zimbabwe,2019,46.781475,30.1,81.40,250000.0,81.50,3.66,0.0,4.58,...,3003.65530,10.47,11760.000230,NaN,-6.144236,1316.740657,38,390757.0,-19.015438,29.154857


Le jeu de données est composé de 3649 observations et 21 variables. Les observations, représentant les individus, sont les données liées à la consommation d'énergie colléctées dans 176 pays et sur 21 années. Théoriquement, on devrait avoir 3696 observations. Il y a donc des données manquantes. Nous allons faire une analyse exploratoire afin de les identifier.

## Analyse exploratoire des données : 

La première étape de l'analyse descriptive est de vérifier l'encodage de nos données. 

In [51]:
energy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3649 entries, 0 to 3648
Data columns (total 21 columns):
 #   Column                                                            Non-Null Count  Dtype  
---  ------                                                            --------------  -----  
 0   Entity                                                            3649 non-null   object 
 1   Year                                                              3649 non-null   int64  
 2   Access to electricity (% of population)                           3639 non-null   float64
 3   Access to clean fuels for cooking                                 3480 non-null   float64
 4   Renewable-electricity-generating-capacity-per-capita              2718 non-null   float64
 5   Financial flows to developing countries (US $)                    1560 non-null   float64
 6   Renewable energy share in the total final energy consumption (%)  3455 non-null   float64
 7   Electricity from fossil fuels (TW

Nous voyons que sur les 21 variables, 2 sont considérées comme qualitatives : Entity et Density. 
Les autres sont considérées comme quantitatives. Cependant l'encodage de Density et Year ne sont pas optimales au vu des observations. En effet, Year devrait être considérée comme qualitative puisque nous n'avons que 21 modalités. De même, Density représentant un scalaire, il serait pertinent de l'encoder comme une variable quantitative.

In [52]:
energy["Entity"]=pd.Categorical(energy["Entity"],ordered=False)
energy["Year"]=pd.Categorical(energy["Year"],ordered=True)
energy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3649 entries, 0 to 3648
Data columns (total 21 columns):
 #   Column                                                            Non-Null Count  Dtype   
---  ------                                                            --------------  -----   
 0   Entity                                                            3649 non-null   category
 1   Year                                                              3649 non-null   category
 2   Access to electricity (% of population)                           3639 non-null   float64 
 3   Access to clean fuels for cooking                                 3480 non-null   float64 
 4   Renewable-electricity-generating-capacity-per-capita              2718 non-null   float64 
 5   Financial flows to developing countries (US $)                    1560 non-null   float64 
 6   Renewable energy share in the total final energy consumption (%)  3455 non-null   float64 
 7   Electricity from fossil 

In [53]:
import re 
def clean_density(density):
    if pd.isnull(density):
        return np.nan
    if isinstance(density, (int,float)): 
        return density
    
    cleaned_density=re.sub(r'[^0-9.]', '', str(density))
    return float(cleaned_density)
energy['Density\\n(P/Km2)']=energy['Density\\n(P/Km2)'].apply(clean_density)



In [54]:
energy.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3649 entries, 0 to 3648
Data columns (total 21 columns):
 #   Column                                                            Non-Null Count  Dtype   
---  ------                                                            --------------  -----   
 0   Entity                                                            3649 non-null   category
 1   Year                                                              3649 non-null   category
 2   Access to electricity (% of population)                           3639 non-null   float64 
 3   Access to clean fuels for cooking                                 3480 non-null   float64 
 4   Renewable-electricity-generating-capacity-per-capita              2718 non-null   float64 
 5   Financial flows to developing countries (US $)                    1560 non-null   float64 
 6   Renewable energy share in the total final energy consumption (%)  3455 non-null   float64 
 7   Electricity from fossil 

In [55]:
energy.head(270)

,Entity,Year,Access to electricity (% of population),Access to clean fuels for cooking,Renewable-electricity-generating-capacity-per-capita,Financial flows to developing countries (US $),Renewable energy share in the total final energy consumption (%),Electricity from fossil fuels (TWh),Electricity from nuclear (TWh),Electricity from renewables (TWh),...,Primary energy consumption per capita (kWh/person),Energy intensity level of primary energy (MJ/$2017 PPP GDP),Value_co2_emissions_kt_by_country,Renewables (% equivalent primary energy),gdp_growth,gdp_per_capita,Density\n(P/Km2),Land Area(Km2),Latitude,Longitude
0,Afghanistan,2000,1.613591,6.2,9.22,20000.0,44.99,0.16,0.0,0.31,...,302.59482,1.64,760.000000,NaN,NaN,NaN,60.0,652230.0,33.93911,67.709953
1,Afghanistan,2001,4.074574,7.2,8.86,130000.0,45.60,0.09,0.0,0.50,...,236.89185,1.74,730.000000,NaN,NaN,NaN,60.0,652230.0,33.93911,67.709953
2,Afghanistan,2002,9.409158,8.2,8.47,3950000.0,37.83,0.13,0.0,0.56,...,210.86215,1.40,1029.999971,NaN,NaN,179.426579,60.0,652230.0,33.93911,67.709953
3,Afghanistan,2003,14.738506,9.5,8.09,25970000.0,36.66,0.31,0.0,0.63,...,229.96822,1.40,1220.000029,NaN,8.832278,190.683814,60.0,652230.0,33.93911,67.709953
4,Afghanistan,2004,20.064968,10.9,7.75,NaN,44.24,0.33,0.0,0.56,...,204.23125,1.20,1029.999971,NaN,1.414118,211.382074,60.0,652230.0,33.93911,67.709953
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,Bahrain,2013,100.000000,100.0,0.90,NaN,0.00,24.36,0.0,0.00,...,161365.53000,9.54,28850.000380,NaN,5.416840,24744.296960,2239.0,765.0,26.06670,50.557700
266,Bahrain,2014,100.000000,100.0,4.26,NaN,0.00,25.62,0.0,0.01,...,153929.60000,8.77,30280.000690,NaN,4.350391,24989.437530,2239.0,765.0,26.06670,50.557700
267,Bahrain,2015,100.000000,100.0,4.15,NaN,0.00,26.77,0.0,0.01,...,149472.80000,9.05,30579.999920,NaN,2.485379,22634.085650,2239.0,765.0,26.06670,50.557700
268,Bahrain,2016,100.000000,100.0,4.69,NaN,0.00,26.80,0.0,0.01,...,144113.94000,8.62,30299.999240,NaN,3.558128,22608.452560,2239.0,765.0,26.06670,50.557700


On a bien le bon format pour toutes nos variables. Passons maintenant à l'identification des valeurs manquantes

In [59]:

energy_missing_value = energy.isnull().sum().sort_values(ascending=False)
print(energy_missing_value)

M=energy.isnull().sum().values
T= np.sum(M)
Percent= np.floor(1/energy.shape[0]*M*100)
#print(M)
print("Taux de valeurs manquantes par variable  en % : ")
print(Percent)
print("Taux global de valeurs manquantes  en % : ")
print(100*T/(energy.shape[0]*energy.shape[1]))





Renewables (% equivalent primary energy)                            2137
Financial flows to developing countries (US $)                      2089
Renewable-electricity-generating-capacity-per-capita                 931
Value_co2_emissions_kt_by_country                                    428
gdp_growth                                                           317
gdp_per_capita                                                       282
Energy intensity level of primary energy (MJ/$2017 PPP GDP)          207
Renewable energy share in the total final energy consumption (%)     194
Access to clean fuels for cooking                                    169
Electricity from nuclear (TWh)                                       126
Low-carbon electricity (% electricity)                                42
Electricity from renewables (TWh)                                     21
Electricity from fossil fuels (TWh)                                   21
Access to electricity (% of population)            

On remarque qu'il y a des données manquantes pour 11 variables. On fait le choix de ne garder que les variables qui ont un taux de données manquantes inférieur à 15%. Nous allons donc supprimer de notre étude les variables : Renewable-electricity-generating-capacity-per-capita, Financial flows to developing
countries (US Dollars) et Renewables (% equivalent primary energy).

In [43]:
#to_drop=['Renewable-electricity-generating-capacity-per-capita', 'Financial flows to developing countries (US $)', 'Renewables (% equivalent primary energy)']
energy=energy.drop(energy.columns[4],axis=1)
#energy=energy
energy.head()

,Entity,Year,Access to electricity (% of population),Access to clean fuels for cooking,Financial flows to developing countries (US $),Renewable energy share in the total final energy consumption (%),Electricity from fossil fuels (TWh),Electricity from nuclear (TWh),Electricity from renewables (TWh),Low-carbon electricity (% electricity),Primary energy consumption per capita (kWh/person),Energy intensity level of primary energy (MJ/$2017 PPP GDP),Value_co2_emissions_kt_by_country,Renewables (% equivalent primary energy),gdp_growth,gdp_per_capita,Density\n(P/Km2),Land Area(Km2),Latitude,Longitude
0,Afghanistan,2000,1.613591,6.2,20000.0,44.99,0.16,0.0,0.31,65.957440,302.59482,1.64,760.000000,NaN,NaN,NaN,60.0,652230.0,33.93911,67.709953
1,Afghanistan,2001,4.074574,7.2,130000.0,45.60,0.09,0.0,0.50,84.745766,236.89185,1.74,730.000000,NaN,NaN,NaN,60.0,652230.0,33.93911,67.709953
2,Afghanistan,2002,9.409158,8.2,3950000.0,37.83,0.13,0.0,0.56,81.159424,210.86215,1.40,1029.999971,NaN,NaN,179.426579,60.0,652230.0,33.93911,67.709953
3,Afghanistan,2003,14.738506,9.5,25970000.0,36.66,0.31,0.0,0.63,67.021280,229.96822,1.40,1220.000029,NaN,8.832278,190.683814,60.0,652230.0,33.93911,67.709953
4,Afghanistan,2004,20.064968,10.9,NaN,44.24,0.33,0.0,0.56,62.921350,204.23125,1.20,1029.999971,NaN,1.414118,211.382074,60.0,652230.0,33.93911,67.709953


In [44]:
energy=energy.drop(energy.columns[4],axis=1)
energy.head()

,Entity,Year,Access to electricity (% of population),Access to clean fuels for cooking,Renewable energy share in the total final energy consumption (%),Electricity from fossil fuels (TWh),Electricity from nuclear (TWh),Electricity from renewables (TWh),Low-carbon electricity (% electricity),Primary energy consumption per capita (kWh/person),Energy intensity level of primary energy (MJ/$2017 PPP GDP),Value_co2_emissions_kt_by_country,Renewables (% equivalent primary energy),gdp_growth,gdp_per_capita,Density\n(P/Km2),Land Area(Km2),Latitude,Longitude
0,Afghanistan,2000,1.613591,6.2,44.99,0.16,0.0,0.31,65.957440,302.59482,1.64,760.000000,NaN,NaN,NaN,60.0,652230.0,33.93911,67.709953
1,Afghanistan,2001,4.074574,7.2,45.60,0.09,0.0,0.50,84.745766,236.89185,1.74,730.000000,NaN,NaN,NaN,60.0,652230.0,33.93911,67.709953
2,Afghanistan,2002,9.409158,8.2,37.83,0.13,0.0,0.56,81.159424,210.86215,1.40,1029.999971,NaN,NaN,179.426579,60.0,652230.0,33.93911,67.709953
3,Afghanistan,2003,14.738506,9.5,36.66,0.31,0.0,0.63,67.021280,229.96822,1.40,1220.000029,NaN,8.832278,190.683814,60.0,652230.0,33.93911,67.709953
4,Afghanistan,2004,20.064968,10.9,44.24,0.33,0.0,0.56,62.921350,204.23125,1.20,1029.999971,NaN,1.414118,211.382074,60.0,652230.0,33.93911,67.709953


In [45]:
energy=energy.drop(energy.columns[12],axis=1)
energy.head()

,Entity,Year,Access to electricity (% of population),Access to clean fuels for cooking,Renewable energy share in the total final energy consumption (%),Electricity from fossil fuels (TWh),Electricity from nuclear (TWh),Electricity from renewables (TWh),Low-carbon electricity (% electricity),Primary energy consumption per capita (kWh/person),Energy intensity level of primary energy (MJ/$2017 PPP GDP),Value_co2_emissions_kt_by_country,gdp_growth,gdp_per_capita,Density\n(P/Km2),Land Area(Km2),Latitude,Longitude
0,Afghanistan,2000,1.613591,6.2,44.99,0.16,0.0,0.31,65.957440,302.59482,1.64,760.000000,NaN,NaN,60.0,652230.0,33.93911,67.709953
1,Afghanistan,2001,4.074574,7.2,45.60,0.09,0.0,0.50,84.745766,236.89185,1.74,730.000000,NaN,NaN,60.0,652230.0,33.93911,67.709953
2,Afghanistan,2002,9.409158,8.2,37.83,0.13,0.0,0.56,81.159424,210.86215,1.40,1029.999971,NaN,179.426579,60.0,652230.0,33.93911,67.709953
3,Afghanistan,2003,14.738506,9.5,36.66,0.31,0.0,0.63,67.021280,229.96822,1.40,1220.000029,8.832278,190.683814,60.0,652230.0,33.93911,67.709953
4,Afghanistan,2004,20.064968,10.9,44.24,0.33,0.0,0.56,62.921350,204.23125,1.20,1029.999971,1.414118,211.382074,60.0,652230.0,33.93911,67.709953


In [46]:
M=energy.isnull().sum().values
T= np.sum(M)
Percent= np.floor(1/energy.shape[0]*M*100)
#print(M)
print("Taux de valeurs manquantes par variable en % : ")
print(Percent)
print("Taux de valeurs manquantes par variable en % : ")
print(100*T/(energy.shape[0]*energy.shape[1]))

Taux de valeurs manquantes par variable en % : 
[ 0.  0.  0.  4.  5.  0.  3.  0.  1.  0.  5. 11.  8.  7.  0.  0.  0.  0.]
Taux de valeurs manquantes par variable en % : 
2.7724490728053346


En enlevant ces colonnes, nous avons drastiquement diminué le taux de valeurs manquantes dans nos données. Ce qui rend notre jeu de données plus exploitable et donc nos prévisions plus pertinentes.